In [32]:
import xml
import xml.etree.ElementTree as ET
import os
from os import path
import glob
from collections import defaultdict, Counter
import numpy as np

from utils import get_ent_info, get_clusters_from_xml

In [33]:
data_dir = "/home/shtoshni/Research/events/data/red/data/source"
source_files = glob.glob("{}/*/*".format(data_dir))

ann_dir = "/home/shtoshni/Research/events/data/red/data/annotation"
ann_files = glob.glob("{}/*/*".format(ann_dir))

output_dir = "/home/shtoshni/Research/events/data/red/error_logs"
if not path.exists(output_dir):
    os.makedirs(output_dir)

## Calculate Types of Coref Chains

In [34]:
parents_type = defaultdict(Counter)
type_properties = defaultdict(Counter)

for source_file in source_files:
    # Read the source doc
    source_lines = open(source_file).readlines()
    source_str = "".join(source_lines)
    
    # Read the annotation file
    base_name = path.basename(source_file)
    dir_name = path.basename(path.dirname(source_file))
    
    ann_file = path.join(path.join(ann_dir, dir_name), base_name + ".RED-Relation.gold.completed.xml")
    
    tree = ET.parse(ann_file)
    root = tree.getroot()
    
    
    for elem in root.iter('relation'):
        parenttype_elem = elem.find('parentsType')
        type_elem = elem.find('type')
        parents_type[parenttype_elem.text][type_elem.text]+= 1
        
        prop_elem = elem.find('properties')
        for tag_elem in prop_elem:
            type_properties[type_elem.text][tag_elem.tag] += 1 
        

for parent_type in parents_type:
    print(parent_type)
    for subtype in parents_type[parent_type]:
        print("\t{}\t{}".format(subtype, parents_type[parent_type][subtype])) 
    print()
print("\n")

for elem_type in type_properties:
    attribute_counts = type_properties[elem_type]
    attribs, attrib_counts = [], []
    for attrib in attribute_counts:
        attribs.append(attrib)
        attrib_counts.append(attribute_counts[attrib])
        
    print(elem_type, *attribs, sep="\t")
    attrib_counts = [''] + attrib_counts
    print(*attrib_counts, sep="\t")
        

TemporalRelations
	TLINK	4209
	ALINK	136

CorefChains
	IDENTICAL	2049
	WHOLE/PART	515
	SET/MEMBER	947
	APPOSITIVE	303
	BRIDGING	383
	REPORTING	596



TLINK	Source	Type	Target	Polarity	ContextualModality	Difficulty
	4209	4209	4209	4209	4209	4209
IDENTICAL	FirstInstance	Coreferring_String
	2049	6512
WHOLE/PART	Whole	Part
	515	815
SET/MEMBER	Set	Member
	947	1614
ALINK	Source	Type	Target	Difficulty
	136	136	136	136
APPOSITIVE	Head	Attribute
	303	303
BRIDGING	Argument	Related_to	FirstInstance	Coreferring_String
	381	433	2	2
REPORTING	Report	Event
	596	1334


# Search For Problematic Spans Marked as Multiple Entities

In [35]:
files_with_issues = defaultdict(list)


for source_file in source_files:
    # Read the source doc
    source_lines = open(source_file).readlines()
    source_str = "".join(source_lines)
    
    # Read the annotation file
    base_name = path.basename(source_file)
    dir_name = path.basename(path.dirname(source_file))
    
    ann_file = path.join(path.join(ann_dir, dir_name), base_name + ".RED-Relation.gold.completed.xml")
    
    tree = ET.parse(ann_file)
    root = tree.getroot()
    
    span_to_elem_id = {}
    # Get info from the XML file
    for elem in root.iter('entity'):
        span_str = list(elem.iter('span'))[0].text
        span_start, span_end = [int(endpoint) for endpoint in span_str.split(",")]
        elem_id = list(elem.iter('id'))[0].text
        elem_type = list(elem.iter('type'))[0].text
        
        span = tuple((span_start, span_end))
        if elem_type == 'ENTITY' or elem_type == 'EVENT':
            if span in span_to_elem_id:
#                 if elem_id == span_to_elem_id[span]:
#                     continue
#                 else:
                files_with_issues[dir_name + "-" + base_name].append(
                    ('Multiple Entities', span, source_str[span_start: span_end],
                     source_str[span_start-15: span_end+15].replace("\n", " "),
                     span_to_elem_id[span][0], span_to_elem_id[span][1], 
                     elem_type, elem_id))
            else:
                span_to_elem_id[span] = (elem_type, elem_id)
            
            
file_issues_list = list(files_with_issues.items())
file_issues_list = sorted(file_issues_list, key=lambda x: x[1], reverse=True)
print("Total files with issues: {}\n".format(len(files_with_issues)))
# for file_name, num_issues in file_issues_list:
#     print(file_name, num_issues)

Total files with issues: 30



# Search For Spans That Are Part of Multiple Chains

In [36]:
# files_with_issues = defaultdict(int)

for source_file in source_files:
    # Read the source doc
    source_lines = open(source_file).readlines()
    source_str = "".join(source_lines)
    
    # Read the annotation file
    base_name = path.basename(source_file)
    dir_name = path.basename(path.dirname(source_file))
    
    ann_file = path.join(path.join(ann_dir, dir_name), base_name + ".RED-Relation.gold.completed.xml")
    
    tree = ET.parse(ann_file)
    root = tree.getroot()
    
    # First build the element ID to span mapping because element IDs are used in CorefChains
    elem_id_to_span = {}
    for elem in root.iter('entity'):
        span_str = list(elem.iter('span'))[0].text
        span_start, span_end = [int(endpoint) for endpoint in span_str.split(",")]
        elem_id = list(elem.iter('id'))[0].text
        elem_type = list(elem.iter('type'))[0].text
        
        span = tuple((span_start, span_end))
        if elem_type == 'ENTITY' or elem_type == 'EVENT':
            if elem_id in elem_id_to_span:
                if span == elem_id_to_span[elem_id]:
                    continue
                else:
                    print(elem_id, span_to_elem_id[span])
            else:
                elem_id_to_span[elem_id] = span
                
    
    span_to_cluster = {}
    cluster_cnter = 0
    for elem in root.iter('relation'):
        type_elem = elem.find('type').text
        if type_elem == 'IDENTICAL':
            cluster_id = elem.find('id').text
            prop_elem = elem.find('properties')
            for sub_elem in prop_elem:
                ent_id = sub_elem.text
                span = elem_id_to_span[ent_id]
                
                if span in span_to_cluster:
                    span_start, span_end = span
                    files_with_issues[dir_name + "-" + base_name].append(
                        ('Multiple Clusters', source_str[span_start: span_end], 
                         source_str[span_start-15: span_end+15].replace("\n", " "), span, ent_id, 
                         span_to_cluster[span], cluster_id))
                else:
                    span_to_cluster[span] = cluster_id
                
            cluster_cnter += 1

file_issues_list = list(files_with_issues.items())
file_issues_list = sorted(file_issues_list, key=lambda x: x[1], reverse=True)
print("Total files with issues: {}\n".format(len(files_with_issues)))
print("Check {} for issues with each file".format(output_dir))
for file_name, issues in file_issues_list:
    with open(path.join(output_dir, file_name + ".txt"), 'w') as f:
        f.write("Number of issues: {}\n\n".format(len(issues)))
        for issue in issues:
            issue_str = [str(issue_attrib) for issue_attrib in issue]
            f.write("   ".join(issue_str) + "\n")
                

Total files with issues: 41

Check /home/shtoshni/Research/events/data/red/error_logs for issues with each file


In [37]:
num_issues_to_num_files = defaultdict(list)
for file_name, issues in file_issues_list:
    num_issues_to_num_files[len(issues)].append(file_name)
    
num_issues = sorted(num_issues_to_num_files.keys())
with open(path.join(output_dir, "meta.txt"), 'w') as f:
    for num_issue in num_issues:
        f.write("Num issues:{}\n".format(num_issue))
        print("Num issues: {}, Num files: {}".format(num_issue, len(num_issues_to_num_files[num_issue])))
        for file_name in num_issues_to_num_files[num_issue]:
            f.write(file_name + "\n")
        f.write("\n")

Num issues: 1, Num files: 17
Num issues: 2, Num files: 10
Num issues: 3, Num files: 3
Num issues: 4, Num files: 2
Num issues: 5, Num files: 2
Num issues: 6, Num files: 2
Num issues: 8, Num files: 1
Num issues: 9, Num files: 1
Num issues: 10, Num files: 1
Num issues: 44, Num files: 1
Num issues: 60, Num files: 1
